<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Quartile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Assign7: MissingValue/MICE+MissForest.xlsx')
df = df.iloc[:, 1:]
df

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L14:WBC,Sex,Age,NATNL,Location,PricipalProblemCode,PricipalProblemName,InPatient,OutPatient,Emergency
0,2299,5812902,-7.158596,100,100,34,0,0,0,0,...,9.410000,หญิง,75,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0,1,0
1,2299,6947220,5.550691,35,0,1,35,0,0,0,...,6.202946,หญิง,76,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0,1,0
2,2299,8186767,-6.063710,100,0,15,0,0,0,0,...,5.420000,หญิง,78,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,239873007,Osteoarthritis of knee,0,1,0
3,2299,9385133,-12.812245,90,0,0,0,0,0,0,...,7.265258,หญิง,79,ไทย,เวชปฏิบัติทั่วไป (OPD)-D03,F411,Generalized anxiety disorder,0,1,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,...,6.704847,หญิง,58,ไทย,หน่วยทะเบียนกลาง(Test พิเศษ)เวชศาสตร์ชันสูตร,None,None,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,6.742022,ชาย,64,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0,1,0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,7.802948,หญิง,72,ไทย,หน่วยคลินิกพิเศษศูนย์บริการทางการแพทย์ชั้นเลิศ,44054006,Type 2 diabetes mellitus,0,1,0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,8.032939,ชาย,67,ไทย,TB (คลินิก)-D707,A150,"Tuberculosis of lung\, confirmed by sputum mic...",0,1,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,18.700000,ชาย,68,ไทย,โรคเลือดและต่อมน้ำเหลือง (คลินิก)-D716,414166008,"Extranodal NK/T-cell lymphoma, nasal type",0,1,0


In [3]:
df = df[['PatientUID','PatientVisitUID','Slopes']]
df

,PatientUID,PatientVisitUID,Slopes
0,2299,5812902,-7.158596
1,2299,6947220,5.550691
2,2299,8186767,-6.063710
3,2299,9385133,-12.812245
4,2304,4916314,28.287500
...,...,...,...
52614,3319961,10553903,10.568365
52615,3319971,10553916,-4.256753
52616,3320048,10572126,-0.692078
52617,3320050,10572565,7.217875


In [4]:
df.isnull().any()

PatientUID         False
PatientVisitUID    False
Slopes             False
dtype: bool

In [13]:
data = pd.DataFrame(df)

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Define the bins
bins = [-80, -5, -0.75, 107]

# Categorize the data
data['Slopes_Category'] = pd.cut(data['Slopes'], bins=bins, labels=['Low', 'Moderate', 'High'])

# Number of splits
n_splits = 5

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store the results
results = []

# Perform the data splitting
for fold, (train_index, test_index) in enumerate(kf.split(data)):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    # Calculate value counts for each category in the fold
    value_counts = train_data['Slopes_Category'].value_counts()
    results.append((fold, len(train_data), value_counts))

    # Display the data in each fold
    print(f"Fold {fold + 1} Train Data:")
    print(train_data)
    print(f"Fold {fold + 1} Test Data:")
    print(test_data)
    print("---------------")

# Display the fold number, number of rows, and value counts for each fold
for i, result in enumerate(results):
    fold_num, fold_size, value_counts = result
    print(f"Fold {fold_num + 1}:")
    print(f"Number of Rows: {fold_size}")
    print(f"Value Counts:\n{value_counts}")
    print("---------------")


Fold 1 Train Data:
       PatientUID  PatientVisitUID     Slopes Slopes_Category
0            2299          5812902  -7.158596             Low
1            2299          6947220   5.550691            High
2            2299          8186767  -6.063710             Low
3            2299          9385133 -12.812245             Low
5            2304          6395251  -8.126415             Low
...           ...              ...        ...             ...
52614     3319961         10553903  10.568365            High
52615     3319971         10553916  -4.256753        Moderate
52616     3320048         10572126  -0.692078            High
52617     3320050         10572565   7.217875            High
52618     3321289         10549658 -32.794557             Low

[42095 rows x 4 columns]
Fold 1 Test Data:
       PatientUID  PatientVisitUID     Slopes Slopes_Category
4            2304          4916314  28.287500            High
6            2304          7900224   0.877756            High
7      

In [18]:
data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
0,2299,5812902,-7.158596,Low
1,2299,6947220,5.550691,High
2,2299,8186767,-6.063710,Low
3,2299,9385133,-12.812245,Low
4,2304,4916314,28.287500,High
...,...,...,...,...
52614,3319961,10553903,10.568365,High
52615,3319971,10553916,-4.256753,Moderate
52616,3320048,10572126,-0.692078,High
52617,3320050,10572565,7.217875,High


## **check**

In [27]:
train_data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
0,2299,5812902,-7.158596,Low
1,2299,6947220,5.550691,High
3,2299,9385133,-12.812245,Low
4,2304,4916314,28.287500,High
5,2304,6395251,-8.126415,Low
...,...,...,...,...
52613,3314119,10421285,-1.342317,Moderate
52614,3319961,10553903,10.568365,High
52615,3319971,10553916,-4.256753,Moderate
52617,3320050,10572565,7.217875,High


In [28]:
test_data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
2,2299,8186767,-6.063710,Low
9,2312,7765456,-7.411572,Low
10,2312,8842427,24.938876,High
12,2354,8438646,4.056631,High
16,2362,8169235,7.488793,High
...,...,...,...,...
52593,3294009,10480935,-4.492308,Moderate
52599,3298298,10281892,7.391940,High
52600,3299347,10328531,-6.835095,Low
52604,3301951,10442151,8.212500,High


# **Excel**

In [ ]:
quartile.to_excel("Quartile.xlsx", index=False)